In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import math
from scipy.stats import norm
import matplotlib.mlab as mlab
import matplotlib.patches as mpatches
from scipy.optimize import curve_fit
from sklearn.metrics import confusion_matrix, f1_score
from sklearn import metrics
import urllib
from sklearn.naive_bayes import BernoulliNB,GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
data = pd.DataFrame(data=cancer.data, columns=cancer.feature_names) # data from library

In [30]:
diagnosis = pd.Series(cancer.target, name= "diagnosis")

In [31]:
dataset = pd.concat([data.iloc[:,0:4], diagnosis], axis=1)

In [38]:
len(diagnosis[diagnosis==1])/len(diagnosis)

0.6274165202108963

In [32]:
dataset.head()

,mean radius,mean texture,mean perimeter,mean area,diagnosis
0,17.99,10.38,122.80,1001.0,0
1,20.57,17.77,132.90,1326.0,0
2,19.69,21.25,130.00,1203.0,0
3,11.42,20.38,77.58,386.1,0
4,20.29,14.34,135.10,1297.0,0
